# Load dependencies

In [1]:
#load dependencies
from sql_conn import connection #SQL connection details and variables
#import neuralnet as nn #functions to form neural network and calculate accuracy and plot accuracy with history
import preproc as pr #functions to preprocess data 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from sklearn import preprocessing #for one-hot encoding

For the Live SQL Server Authentication, please enter your


# SQL Script

In [7]:
s='''
    
        SELECT DISTINCT
        [StudyTitle]
        ,[StudyShortName]
        ,StudyResearchSummary
        ,StudyInclusionCriteria
        ,StudyDesignType
        ,StudyPhases_Concatenated
        ,StudySettings_Concatenated
        ,StudyManagingSpecialty
        ,StudyManagingSpecialty_PrimarySubSpecialty
        ,[ICDChapter]

        FROM(
        SELECT 
        [StudyID]
        ,[ICDChapter]
        from [CPMS_BI].[dbo].[ODP_StudyICD]
        WHERE StudyID is NOT NULL) AS IC

        LEFT JOIN 
        (SELECT 
        StudyID
        ,[StudyTitle]
        ,[StudyShortName]
        ,StudyResearchSummary
        ,StudyInclusionCriteria
        ,StudyDesignType
        ,StudyPhases_Concatenated
        ,StudySettings_Concatenated
        ,StudyManagingSpecialty
        ,StudyManagingSpecialty_PrimarySubSpecialty
        ,Study_IsCommercial
        ,Study_IsDraft

        FROM [CPMS_BI].[dbo].[ODP_Study]
        WHERE 1=1
        ) AS ST
        ON IC.StudyID = ST.StudyID
        WHERE 1=1
        AND Study_IsCommercial = 0
        AND Study_IsDraft=0


        UNION

        SELECT  DISTINCT
        [StudyTitle]
        ,[StudyShortName]
        ,StudyResearchSummary
        ,StudyInclusionCriteria
        ,StudyDesignType
        ,StudyPhases_Concatenated
        ,StudySettings_Concatenated
        ,StudyManagingSpecialty
        ,StudyManagingSpecialty_PrimarySubSpecialty
        ,'' as [ICDChapter]


        FROM [CPMS_BI].[dbo].[ODP_Study]
        WHERE 1=1
        AND Study_IsCommercial = 0
        AND Study_IsDraft=0
        AND StudyID NOT IN (
        SELECT DISTINCT StudyID
        FROM [CPMS_BI].[dbo].[ODP_StudyICD]
        WHERE StudyID is NOT NULL
        )
    '''

# with open("C:/Users/busaji/Documents/Projects/StudyClassifier-main/dat/sql-read.txt","r") as f:
#     s = f.read()

In [8]:
study_df = pd.read_sql(s, connection)

C:\Users\busaji\AppData\Local\Temp\ipykernel_21292\3074574563.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  study_df = pd.read_sql(s, connection)


In [9]:
study_df.dtypes

StudyTitle                                    object
StudyShortName                                object
StudyResearchSummary                          object
StudyInclusionCriteria                        object
StudyDesignType                               object
StudyPhases_Concatenated                      object
StudySettings_Concatenated                    object
StudyManagingSpecialty                        object
StudyManagingSpecialty_PrimarySubSpecialty    object
ICDChapter                                    object
dtype: object

In [10]:
#pre-processing
study_df2 = study_df.fillna("not available")

#convert to float
#study_df['StudyResearchSummary'] = study_df['StudyResearchSummary'].astype("string")

In [11]:
study_df2

,StudyTitle,StudyShortName,StudyResearchSummary,StudyInclusionCriteria,StudyDesignType,StudyPhases_Concatenated,StudySettings_Concatenated,StudyManagingSpecialty,StudyManagingSpecialty_PrimarySubSpecialty,ICDChapter
0,​Maximising the use of community digital asset...,Digital Assets Study,The study will be conducted in two phases. Pha...,Participants in phase 1 and phase 2 must be ad...,Observational,N/A,"Community, None of the above",Mental Health,Service evaluations - service users,Mental and behavioural disorders
1,"""Being-neurodivergent-in-the-world"": an embodi...","""Being-neurodivergent-in-the-world"": an embodi...",This neurodivergent-led (auto)ethnography focu...,"- Adults (+18 years of age, at the time of app...",Observational,N/A,Secondary Care,Mental Health,Developmental Disorders,Mental and behavioural disorders
2,"""pErsonalised Nocebo Assessment of Beta-bLockE...",ENABLE-HF ver1.0,Heart failure (a problem with pumping function...,1. Having heart failure with reduced ejection ...,Observational,N/A,"Secondary Care, Tertiary Care",Cardiovascular Disease,Heart Failure,
3,"""Real world"" pain outcomes and experiences of ...",MIDAS-GP [Patient facing name: MIDAS-GP Study],This study is one component of the MIDAS progr...,- Patients aged 18 years and above - Registe...,Observational,N/A,Primary Care,Primary Care,Musculoskeletal Disorders,"Symptoms, signs and abnormal clinical and labo..."
4,"""RECOGNISE” – taRgeted intermittEnt gluCose mO...",RECOGNISE - a feasibility study,Aim: Is it possible to carry out a large-sca...,Clinical Feasibility Study Women will be in...,Both,N/A,Tertiary Care,Reproductive Health and Childbirth,Maternal Medicine,"Pregnancy, childbirth and the puerperium"
...,...,...,...,...,...,...,...,...,...,...
27693,ZERO radiation exposure for catheter ablation ...,Non-fluoroscopy ablation of AF/AT (ZERO-AF),Atrial fibrillation is the most common arrhyth...,• Any type of atrial fibrillation or left atri...,Interventional,N/A,Tertiary Care,Cardiovascular Disease,Arrhythmia,Diseases of the circulatory system
27694,Zinc and inflammation in sepsis,Zinc and sepsis v1,,1) Patients with sepsis. Clinical suspicion o...,Observational,N/A,Secondary Care,Infection,Infectious diseases and microbiology (migration),
27695,Zn-DDC to target hypoxia-NFkappaB-CSCs pathway...,Zn-DDC to target hypoxia-NFkappaB-CSCs pathway...,The outlook for patients with haematological m...,• Patients diagnosed with haematological malig...,Observational,N/A,Secondary Care,Cancer,Haematological Oncology,Neoplasms
27696,Zoledronate in the Prevention of Paget's - Lon...,Zoledronate in the Prevention of Paget's - Lon...,Paget's disease of the bone (PDB) is a metabol...,- Previous participation in the ZiPP study - ...,Observational,N/A,Secondary Care,Musculoskeletal Disorders,Other,


In [51]:
#workaround - but 'reload' is not defined
import sys
reload(sys)
sys.setdefaultencoding('utf8')

NameError: name 'reload' is not defined

1. Convert to string

In [60]:
#study_df2['StudyResearchSummary'] = study_df2['StudyResearchSummary'].encode('utf-8')

for index, row in study_df2.iterrows():
    row['StudyResearchSummary'] = row['StudyResearchSummary'].encode('utf-8')




# TF-IDF Vectorizer

In [61]:

corpus = study_df2['StudyResearchSummary']
vectorizer = TfidfVectorizer(lowercase = True, encoding = "utf-8", decode_error="ignore", stop_words='english', analyzer='word', ngram_range=(1,2))
X = vectorizer.fit_transform(corpus)
vectorizer.get_feature_names_out()
print(X.shape)

(19513, 956142)


In [62]:
X.head()

AttributeError: 'csr_matrix' object has no attribute 'head'

# K-Means Clustering

In [63]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=20, random_state=0, n_init="auto").fit(X)


In [64]:
feat_nms = kmeans.get_feature_names_out()

In [65]:
feat_nms

array(['kmeans0', 'kmeans1', 'kmeans2', 'kmeans3', 'kmeans4', 'kmeans5',
       'kmeans6', 'kmeans7', 'kmeans8', 'kmeans9', 'kmeans10', 'kmeans11',
       'kmeans12', 'kmeans13', 'kmeans14', 'kmeans15', 'kmeans16',
       'kmeans17', 'kmeans18', 'kmeans19'], dtype=object)

In [67]:
kmeans.

KMeans(n_clusters=20, random_state=0)

# LDA

In [66]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.datasets import make_multilabel_classification

lda = LatentDirichletAllocation(n_components=5,
    random_state=0)
lda.fit(X) #fit X


LatentDirichletAllocation(n_components=5, random_state=0)

In [68]:
#transform X
X_lda =lda.transform(X)



In [71]:
X_lda.shape

(19513, 5)

In [ ]:
#get output as pandas
X_lda.set_output(*,transform="pandas")